In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

/home/arjun/AI_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Dataset

In [3]:
animals = {0: 'butterfly', 1: 'cat', 2: 'cow', 3: 'dog', 4: 'elephant', 5: 'hen', 6: 'horse', 7: 'sheep', 8: 'spider', 9: 'squirrel'}
rev_animals = {j:i for i,j in animals.items()}
print(rev_animals)

{'butterfly': 0, 'cat': 1, 'cow': 2, 'dog': 3, 'elephant': 4, 'hen': 5, 'horse': 6, 'sheep': 7, 'spider': 8, 'squirrel': 9}


In [4]:
path_to_main_dir = '/home/arjun/Desktop/Datasets/animals'
X,y = [], []
for sub in os.listdir(path_to_main_dir):
    for img in os.listdir(os.path.join(path_to_main_dir,sub))[:1000]:
        image = cv2.imread(os.path.join(os.path.join(path_to_main_dir,sub), img))
        x = cv2.resize(image, (100,100))
        X.append(x)
        y.append(rev_animals[sub])

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [5]:
len(X)

10000

In [6]:
X = torch.tensor(X).to(device).float()
y = np.array(y)
y = F.one_hot(torch.tensor(y), num_classes=10).to(device).float()

/tmp/ipykernel_71646/2460507321.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X = torch.tensor(X).to(device).float()


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.8)

### Converting to DataLoader

In [8]:
class Dataset:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [9]:
train = Dataset(X_train, y_train)
test = Dataset(X_test, y_test)
len(train), len(test)

(8000, 2000)

In [10]:
train_dataloader = DataLoader(train, 1000, True)
test_dataloader = DataLoader(test, 1000, True)

# Creating Model

In [11]:
class MLP(nn.Module):
    def __init__(self, inp_size, hid1, hid2, hid3, out_size):
        super(MLP, self).__init__()
        
        self.lay1 = nn.Linear(inp_size, hid1)
        self.lay2 = nn.ReLU()
        self.lay3 = nn.Linear(hid1, hid2)
        self.lay4 = nn.ReLU()
        self.lay5 = nn.Linear(hid2, hid3)
        self.lay6 = nn.ReLU()
        self.lay7 = nn.Linear(hid3, out_size)

    def forward(self, X):
        out = self.lay1(X)
        out = self.lay2(out)
        out = self.lay3(out)
        out = self.lay4(out)
        out = self.lay5(out)
        out = self.lay6(out)
        out = self.lay7(out)
        return out

model = MLP(100*100*3, 500, 100, 100, 10).to(device)
model

MLP(
  (lay1): Linear(in_features=30000, out_features=500, bias=True)
  (lay2): ReLU()
  (lay3): Linear(in_features=500, out_features=100, bias=True)
  (lay4): ReLU()
  (lay5): Linear(in_features=100, out_features=100, bias=True)
  (lay6): ReLU()
  (lay7): Linear(in_features=100, out_features=10, bias=True)
)

In [12]:
lossCat = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=1e-5)

# Training

In [13]:

for epoch in range(200):
    for step, (x,y) in enumerate(train_dataloader):
        # forward pass
        x = x.reshape(-1, 100*100*3)
        y_pred = model(x)
        
        # Loss calculation
        loss = lossCat(y_pred, y)
        
        # Backpropogation
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
    
    if epoch%10 == 0:    
        print('Epoch:', epoch, ':', loss.item())

Epoch: 0 : 4.086862087249756
Epoch: 10 : 2.122422933578491
Epoch: 20 : 1.9554004669189453
Epoch: 30 : 1.7912237644195557
Epoch: 40 : 1.6890054941177368
Epoch: 50 : 1.5459880828857422
Epoch: 60 : 1.4371079206466675
Epoch: 70 : 1.3451910018920898
Epoch: 80 : 1.2515283823013306
Epoch: 90 : 1.1741136312484741
Epoch: 100 : 1.042250394821167
Epoch: 110 : 0.919402003288269
Epoch: 120 : 0.841399073600769
Epoch: 130 : 0.7646727561950684
Epoch: 140 : 0.7396352291107178
Epoch: 150 : 0.6022002100944519
Epoch: 160 : 0.5676324963569641
Epoch: 170 : 0.5065752863883972
Epoch: 180 : 0.4671515226364136
Epoch: 190 : 0.437844455242157


# Model Evaluation

### Train Data

In [14]:
tot, correct = 0, 0
for x,y in train_dataloader:
    x = x.reshape(-1, 100*100*3)
    y_pred = model(x)
    # print(torch.argmax(y_pred,dim=1))
    # print(torch.argmax(y,dim=1))
    correct += sum(torch.argmax(y_pred,dim=1) == torch.argmax(y,dim=1))
    tot += len(y)

print("Accuracy:", (correct/tot*100).item(),'%')
tot
    

Accuracy: 95.75000762939453 %


8000

### Test Data

In [15]:
tot, correct = 0, 0
for x,y in test_dataloader:
    x = x.reshape(-1, 100*100*3)
    y_pred = model(x)
    # print(torch.argmax(y_pred,dim=1))
    # print(torch.argmax(y,dim=1))
    correct += sum(torch.argmax(y_pred,dim=1) == torch.argmax(y,dim=1))
    tot += len(y)

print("Accuracy:", (correct/tot*100).item(),'%')
tot

Accuracy: 33.29999923706055 %


2000

Best Accuracy: 39% :(